# Training DNQ for Tic Tac Toe and play against LLM

Based on [RL against random policy opponent with PettingZoo](https://tianshou.org/en/stable/01_tutorials/04_tictactoe.html).

In [1]:
%load_ext autoreload
%autoreload 2

# Install dependencies

In [2]:
!pip install gymnasium==0.29.1 pygame==2.3.0 pettingzoo==1.24.3 tianshou==0.5.1 transformers==4.39.1 accelerate==0.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 847.8/847.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (1

# Import needed dependencies

In [39]:
from typing import Any, Dict
import gymnasium as gym
from gymnasium.spaces import Discrete, Space
import torch
from torch import nn
import numpy as np
import re
from pettingzoo.classic import tictactoe_v3
from torch.utils.tensorboard import SummaryWriter

from tianshou.data import Batch, Collector, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.env.pettingzoo_env import PettingZooEnv
from tianshou.policy import (
    BasePolicy,
    DQNPolicy,
    MultiAgentPolicyManager,
    RandomPolicy,
)
from tianshou.trainer import OffpolicyTrainer
from tianshou.utils import TensorboardLogger
from tianshou.utils.net.common import Net

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Setup environment

In [40]:
def get_env(render_mode=None):
  return PettingZooEnv(tictactoe_v3.env(render_mode=render_mode))

# create the environment and get the shape of the states and shape of the actions
env = get_env()
observation_space = env.observation_space['observation'] if isinstance(
  env.observation_space, gym.spaces.Dict
) else env.observation_space
state_shape = observation_space.shape or observation_space.n
action_shape = env.action_space.shape or env.action_space.n

# Setup policies for training DNQPolicy

One training policy (DNQPolicy) and the opponent (RandomPolicy).

In [41]:
# Hidden sizes: shape of the MLP
hidden_sizes = [128, 128, 128, 128]
# device to train on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# the number of steps to look ahead
estimation_step = 3
# the target network update frequency (0 if you do not use the target network).
target_update_freq = 320
# learning rate of the Adam optimizer
lr = 1e-4

In [42]:
def get_dnq_policy():
  # The deep learning model (MLP) that underpins the behaviour of the agent (it is not the agent itself).
  net = Net(
    state_shape,
    action_shape,
    hidden_sizes=hidden_sizes,
    device=device
  ).to(device)

  # Adam optimizer
  optim = torch.optim.Adam(net.parameters(), lr=lr)

  # Agent to learn
  return DQNPolicy(
    model=net,
    optim=optim,
    action_space=env.action_space,
    estimation_step=estimation_step,
    target_update_freq=target_update_freq
  )

In [43]:
agent_learn = get_dnq_policy()
agent2 = RandomPolicy()

agents = [agent_learn, agent2]

# Multi-agent policy manager for Multi-Agent Reinforcement Learning (https://tianshou.org/en/stable/01_tutorials/07_cheatsheet.html#marl-example)
policy = MultiAgentPolicyManager(agents, env)

# Train the agent with DNQ
Using the OffPolicyTrainer.

In [44]:
# number of training environments
training_num = 100
# number of testing environments
test_num = 100
# size of the VectorReplayBuffer
buffer_size = 20000
# the batch size of sample data, which is going to feed in the policy network.
batch_size = 64
# the maximum number of epochs for training. The training process might be finished before reaching max_epoch if stop_fn is set.
epoch = 100
# the number of transitions collected per epoch.
step_per_epoch = 1000
# the number of transitions the collector would collect before the network update,
# i.e., trainer will collect "step_per_collect" transitions and do some policy network update
# repeatedly in each epoch.
step_per_collect = 10
# used in the stop function when the mean rewards are over this threshold
win_rate = 0.95
# The eps for epsilon-greedy exploration for test and training
eps_test = 0.05
eps_train = 0.1
# the number of times the policy network would be updated per transition after (step_per_collect)
# transitions are collected, e.g., if update_per_step set to 0.3, and step_per_collect is 256,
# policy will be updated round(256 * 0.3 = 76.8) = 77 times after 256 transitions are collected
# by the collector. Default to 1.
update_per_step = 0.1
# path to save the results and the logging
path = '/content/tic_tac_toe/dnq'

# Dummy vectorized environment wrapper, implemented in for-loop.
# This has the same interface as true vectorized environment, but the rollout does not happen in parallel.
# So, all workers just wait for each other and the environment is as efficient as using a single environment.
# This can be useful for testing or for demonstration purposes.
train_envs = DummyVectorEnv([get_env for _ in range(training_num)])
test_envs = DummyVectorEnv([get_env for _ in range(test_num)])

# VectorReplayBuffer contains n ReplayBuffer with the same size.
# It is used for storing transition from different environments yet keeping the order of time.
vectorReplayBuffer = VectorReplayBuffer(buffer_size, len(train_envs))

# determine whether the action needs to be modified with corresponding policy’s exploration noise.
# If so, “policy. exploration_noise(act, batch)” will be called automatically to add the
# exploration noise into action.
exploration_noise = True

# Train and test collector
# Collector enables the policy to interact with different types of envs with exact number of steps or episodes.
train_collector = Collector(policy, train_envs, vectorReplayBuffer, exploration_noise=exploration_noise)
test_collector = Collector(policy, test_envs, exploration_noise=exploration_noise)

# Collect a specified number of step or episode.
train_collector.collect(n_step=batch_size * training_num)

# A logger that logs statistics during training/testing/updating
writer = SummaryWriter(path)
logger = TensorboardLogger(writer)

# ID of the agent that we are training
agent_id = 'player_1'

# Functions for the OffpolicyTrainer
# Save the best model
def save_best_fn(policy):
  torch.save(policy.policies[agent_id].state_dict(), path + '/policy.pth')

# When to stop training
def stop_fn(mean_rewards):
    return mean_rewards >= win_rate

# a hook called at the beginning of training in each epoch. It can be used to perform custom additional operations
def train_fn(epoch, env_step):
    # Set the eps for epsilon-greedy exploration.
    policy.policies[agent_id].set_eps(eps_train)

def test_fn(epoch, env_step):
    # Set the eps for epsilon-greedy exploration.
    policy.policies[agent_id].set_eps(eps_test)

# A function with signature used in multi-agent RL.
# We need to return a single scalar for each episode’s result to monitor training in the multi-agent RL setting.
# This function specifies what is the desired metric, e.g., the reward of agent 1 or the average reward over all agents.
def reward_metric(rews):
    return rews[:, 0]

# Offpolicy trainer, samples mini-batches from buffer and passes them to update.
result = OffpolicyTrainer(
  policy,
  train_collector,
  test_collector,
  epoch,
  step_per_epoch,
  step_per_collect,
  test_num,
  batch_size,
  train_fn=train_fn,
  test_fn=test_fn,
  stop_fn=stop_fn,
  save_best_fn=save_best_fn,
  update_per_step=update_per_step,
  logger=logger,
  test_in_train=False, # whether to test in the training phase.
  reward_metric=reward_metric
).run()

Epoch #1: 1001it [00:01, 943.22it/s, env_step=1000, len=7, n/ep=11, n/st=100, player_1/loss=0.339, rew=0.36]                          


Epoch #1: test_reward: 0.060000 ± 0.957288, best_reward: 0.830000 ± 0.510979 in #0


Epoch #2: 1001it [00:01, 765.39it/s, env_step=2000, len=7, n/ep=14, n/st=100, player_1/loss=0.182, rew=-0.29]                          


Epoch #2: test_reward: -0.270000 ± 0.936536, best_reward: 0.830000 ± 0.510979 in #0


Epoch #3: 1001it [00:01, 773.91it/s, env_step=3000, len=7, n/ep=17, n/st=100, player_1/loss=0.191, rew=-0.12]                          


...

Epoch #12: test_reward: 0.560000 ± 0.816333, best_reward: 0.900000 ± 0.387298 in #7


Epoch #13: 1001it [00:01, 895.64it/s, env_step=13000, len=6, n/ep=18, n/st=100, player_1/loss=0.137, rew=0.39]                          


Epoch #13: test_reward: 0.700000 ± 0.685565, best_reward: 0.900000 ± 0.387298 in #7


Epoch #14: 1001it [00:01, 930.28it/s, env_step=14000, len=6, n/ep=14, n/st=100, player_1/loss=0.134, rew=0.86]                          


Epoch #14: test_reward: 0.970000 ± 0.170587, best_reward: 0.970000 ± 0.170587 in #14


# Play DNQ trained agent against random player

Play with the trained agent to the opponent a number of episodes and print the results

In [46]:
env = get_env(render_mode=None)
env = DummyVectorEnv([lambda: env])
collector = Collector(policy, env, exploration_noise=True)
# play number of episodes
result = collector.collect(n_episode=100, render=None)
rews, lens = result["rews"], result["lens"]
print(f"Final reward: {rews[:, 0].mean()}, length: {lens.mean()}")

won = 0
draw = 0
lost = 0
for res in result['rews']:
  if res[0] == 1:
    won += 1
  elif res[0] == -1:
    lost +=1
  else:
    draw += 1

print("Win: " + str(won) + " lost: " + str(lost) + " draw: " + str(draw))

Final reward: 0.88, length: 5.62
Win: 92 lost: 4 draw: 4


# Setup LLM Agent

In [47]:
class LLMAgent(BasePolicy):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        models_dict = {"StableLM Zephyr 3B": "stabilityai/stablelm-zephyr-3b"}
        model_id = models_dict["StableLM Zephyr 3B"]
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)#, device=0)
        self.pipe = pipeline("text-generation", model=model_id, device_map="auto", tokenizer=self.tokenizer, torch_dtype=torch.bfloat16)


    def field_to_string(self, field, c):
      return_value = "";
      if field[0] == 1:
        return_value = "X "
      elif field[1] == 1:
          return_value = "O "
      else:
          return_value = str(c) + " "
      if c < 6:
          return_value += "| "
      return return_value

    def board_to_string(self, batch: Batch) -> str:
        rows = batch.obs['obs']
        row1 = self.field_to_string(rows[0][0][0], 0) + self.field_to_string(rows[0][1][0], 3) + self.field_to_string(rows[0][2][0], 6)
        row2 = self.field_to_string(rows[0][0][1], 1) + self.field_to_string(rows[0][1][1], 4) + self.field_to_string(rows[0][2][1], 7)
        row3 = self.field_to_string(rows[0][0][2], 2) + self.field_to_string(rows[0][1][2], 5) + self.field_to_string(rows[0][2][2], 8)
        return row1 + '\n' + row2 + '\n' + row3

    def ask_llm_for_choice(self, board: str, possible_choices: str) -> int:
        print(board)
        self.job_description = "You will be provided with a tic tac toe board. There are two players, X and O. An empty board looks likes this:\n0 3 6\n1 4 7\n2 5 8\nWhen a player made a move a X or O is placed on the board.\nYou are player X and should choose the best possible option. The possible numbers are " + possible_choices + ". Only answer best number to choose, no comments or explanation, just a number."
        output = self.generate(board,0.7,15)
        print("Output: " + output)

        extracted_choice = [int(i) for i in re.sub(r'[^0-9\s]', '', output).split() if i.isdigit()]
        for n in extracted_choice:
          print("extracted_choice: " + str(n))
        if extracted_choice:
          print("Choice: " + str(extracted_choice[0]))
          return extracted_choice[0]
        else:
          print("No choice, try "+str(possible_choices[0]))
          return int(possible_choices[0])

    def generate(self, question, temperature=0.7, max_new_tokens=512):
        #return "left"
        messages = [
            {
                "role": "system",
                "content": self.job_description,
            },
        ]
        messages.append({"role": "user", "content": question})
        prompt = self.pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        outputs = self.pipe(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_k=50, top_p=0.95)
        output = outputs[0]["generated_text"]
        messages.append({"role": "assistant", "content": output})
        #print(output)
        response_start = output.rfind('<|assistant|>')
        text_output = output[response_start + len('<|assistant|>'):]
        return text_output

    def forward(
        self, batch: Batch, state: dict | Batch | np.ndarray | None = None
    ) -> Batch:
        all_choices = np.array(list(range(0,9)))

        board = self.board_to_string(batch)

        mask = batch.obs.mask.flatten()
        masked_choices = all_choices[mask.astype(bool)].astype(str)
        masked_choices_text = ", ".join(masked_choices)

        if len(masked_choices) == 1:
          print("Only one option: " + masked_choices[0])
          return Batch(act=[int(masked_choices[0])])

        choice = -1
        tries = 3
        while str(choice) not in masked_choices and tries > 0:
          print("try: " + str(tries))
          choice = self.ask_llm_for_choice(board, masked_choices_text)
          tries -= 1

        return Batch(act=[choice])

    def learn(self, batch: Batch) -> Dict[str, Any]:
        return {}

# Play LLM against DNQ

In [48]:
env = get_env(render_mode=None)
llm_agent = LLMAgent()
agents = [agent_learn, llm_agent]
policy = MultiAgentPolicyManager(agents, env)

env = DummyVectorEnv([lambda: env])
collector = Collector(policy, env, exploration_noise=True)
# play number of episodes
result = collector.collect(n_episode=100, render=None)
rews, lens = result["rews"], result["lens"]
print(f"Final reward: {rews[:, 0].mean()}, length: {lens.mean()}")

won = 0
draw = 0
lost = 0
for res in result['rews']:
  if res[0] == 1:
    won += 1
  elif res[0] == -1:
    lost +=1
  else:
    draw += 1

print("Win: " + str(won) + " lost: " + str(lost) + " draw: " + str(draw))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


try: 3
0 | 3 | 6 
1 | O | 7 
2 | 5 | 8 
Output: 
5
extracted_choice: 5
Choice: 5
try: 3
0 | 3 | 6 
O | O | 7 
2 | X | 8 
Output: 
You should choose the number 8.
extracted_choice: 8
Choice: 8
try: 3
0 | 3 | 6 
1 | O | 7 
2 | 5 | 8 
Output: 
5
extracted_choice: 5
Choice: 5
try: 3
0 | 3 | 6 
O | O | 7 
2 | X | 8 
Output: 
The best number for player X to choose is 8.
extracted_choice: 8
Choice: 8
try: 3
0 | 3 | 6 
1 | O | 7 
2 | 5 | 8 
Output: 
Player X should choose the number 6.
extracted_choice: 6
Choice: 6
try: 3
0 | 3 | X 
O | O | 7 
2 | 5 | 8 
Output: 
You should choose the number 3.
extracted_choice: 3
Choice: 3
try: 3
0 | 3 | 6 
1 | O | 7 
2 | 5 | 8 
Output: 
5
extracted_choice: 5
Choice: 5
try: 3
0 | 3 | 6 
O | O | 7 
2 | X | 8 
Output: 
Player X should choose the number 6.
extracted_choice: 6
Choice: 6
try: 3
0 | 3 | 6 
1 | O | 7 
2 | 5 | 8 
Output: 
Player X should choose number 3.
extracted_choice: 3
Choice: 3
try: 3
0 | X | 6 
O | O | 7 
2 | 5 | 8 
Output: 
5
extracted_choice